<a href="https://colab.research.google.com/github/crazy-lazy-life/Reinforcement_Learning_School_of_AI/blob/master/Value_%26_Policy_Iteration_RL_School_Of_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import gym
from gym import wrappers

In [0]:
def run_episode(env, policy, gamma = 0.9, render = False):
  obs = env.reset()
  total_reward = 0
  step_index = 0
  while True:
    if render:
      env.render()
    obs, reward, done, _ = env.step(int(policy[obs]))
    total_reward += (gamma ** step_index * reward)
    step_index += 1
    if done:
      break
   
  return total_reward

In [0]:
def evaluate_policy(env, policy, gamma = 0.9, n = 100):
  scores = [run_episode(env, policy, gamma = gamma, render = False) for _ in range(n)]
  return np.mean(scores)

In [0]:
def extract_policy(v, gamma = 0.9):
  policy = np.zeros(env.env.nS)
  for s in range(env.env.nS):
    q_sa = np.zeros(env.action_space.n)
    for a in range(env.action_space.n):
      for next_sr in env.env.P[s][a]:
        p, s_, r, _ = next_sr
        q_sa[a] += (p * (r + gamma * v[s_]))
       
    policy[s] = np.argmax(q_sa)
    
  return policy

In [0]:
def compute_policy_v(env, policy, gamma=1.0):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

In [0]:
def value_iteration(env, gamma = 0.9):
  v = np.zeros(env.env.nS)
  max_iterations = 10000
  eps = 1e-20
  for i in range(max_iterations):
    prev_v = np.copy(v)
    for s in range(env.env.nS):
      q_sa = [sum([p*(r+prev_v[s_]) for p, s_, r, _ in env.env.P[s][a]]) for a in range(env.env.nA)]
      v[s] = max(q_sa)

    if (np.sum(np.fabs(prev_v - v)) <= eps):
      break;
     
  return v

In [0]:
def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    policy = np.random.choice(env.env.nA, size=(env.env.nS))  # initialize a random policy
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy


In [72]:
if __name__ == '__main__':
  env_name = 'FrozenLake8x8-v0'
  gamma = 0.9
  env = gym.make(env_name)
  print('By Value Iteration')
  optimal_v = value_iteration(env, gamma)
  policy = extract_policy(optimal_v, gamma)
  policy_score = evaluate_policy(env, policy, gamma, n=1000)
  print('Policy average score = ', policy_score)
  print('By Policy Iteration')
  optimal_policy = policy_iteration(env, gamma = 1.0)
  scores = evaluate_policy(env, optimal_policy, gamma = 1.0)
  print('Average scores = ', np.mean(scores))

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


By Value Iteration
Policy average score =  0.002189046980988554
By Policy Iteration
Policy-Iteration converged at step 6.
Average scores =  0.89
